In [1]:
import pandas as pd
import numpy as np

In [9]:
# 读取数据
output_file = 'D:\\FutureData\\ricequant\\1d_2017to2024_noadjust.h5'
df0 = pd.read_hdf(output_file, key='/MA').reset_index()
df1 = pd.read_hdf(output_file, key='/PP').reset_index()


In [17]:
import numpy as np

class KalmanFilter:
    def __init__(self):
        self.x = np.array([1.0])  # 初始系数（假设1:1配比）
        self.P = np.eye(1)        # 状态协方差
        self.Q = 0.01             # 过程噪声
        self.R = 0.1              # 观测噪声

    def update(self, z):
        # 预测步骤
        x_pred = self.x
        P_pred = self.P + self.Q

        # 更新步骤
        K = P_pred / (P_pred + self.R)
        self.x = x_pred + K * (z - x_pred)
        self.P = (1 - K) * P_pred
        return self.x[0]

def kalman_ratio(df1, df2):
    kf = KalmanFilter()
    spreads = []
    for p1, p2 in zip(df1, df2):
        if p2 != 0:
            ratio = p1 / p2  # 实时价格比
            beta = kf.update(ratio)
        spreads.append(p1 - beta * p2)

    # 取末段均值确定整数配比
    final_beta = np.mean(kf.x[-30:]) if len(df1) >30 else round(kf.x[-1])
    return simplify_ratio(final_beta), np.array(spreads)

kalman_ratio(df1.close,df0.close)

((25, 9),
 array([[531.71171171],
        [241.63814015],
        [180.71105   ],
        ...,
        [-31.38092881],
        [-81.91067103],
        [-57.64115829]]))

In [14]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
def simplify_ratio(ratio, max_denominator=10):
    """
    将浮点比例转换为最简整数比
    :param ratio: 浮点比例值
    :param max_denominator: 最大允许的分母值
    :return: (分子, 分母) 的元组
    """
    from fractions import Fraction
    frac = Fraction(ratio).limit_denominator(max_denominator)
    return (frac.numerator, frac.denominator)

def cointegration_ratio(df1, df2):

    # 协整回归
    X = sm.add_constant(df2)
    model = sm.OLS(df1, X).fit()
    beta = model.params[1] # 回归系数整数化
    spread = df1 - beta * df2  # 价差序列

    return simplify_ratio(beta), spread  # 配比格式(资产1单位:资产β单位)

cointegration_ratio(df1.close,df0.close)

C:\Users\flower\AppData\Local\Temp\ipykernel_8772\3933633177.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = model.params[1] # 回归系数整数化


((13, 8),
 0       4250.271689
 1       4208.413727
 2       4242.987613
 3       4153.277424
 4       4246.924975
            ...     
 1938    3105.942179
 1939    3112.561500
 1940    3099.851310
 1941    3062.936444
 1942    3099.976144
 Name: close, Length: 1943, dtype: float64)